In [0]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, TimestampType
import pyspark.sql.functions as F
import json
from pyspark.sql.window import Window

In [0]:
display(dbutils.fs.ls('/Volumes/gizmobox/landing/operational_data/memberships/'))

In [0]:
df = spark.read.format('binaryFile').load('/Volumes/gizmobox/landing/operational_data/memberships/*/*.png')
display(df)

###1.Create Memberships table in bronze Schema

In [0]:
df.writeTo('gizmobox.bronze.py_memberships').createOrReplace()

###2. Read from csv files

In [0]:
df_addresses = (
    spark.read.format('csv')
    .option("header", "true")
    .option("delimiter", "\t")
    .load('/Volumes/gizmobox/landing/operational_data/addresses/')
)
display(df_csv)

###3. Create Addresses table in the Bronze Layer

In [0]:
df_addresses.writeTo('gizmobox.bronze.py_addresses').createOrReplace()

In [0]:
%sql
SELECT * FROM gizmobox.bronze.py_addresses

###4. Read from Payment csv files

In [0]:
df_payment_schema = (
    StructType([
        StructField('payment_id', IntegerType()),
        StructField('order_id', IntegerType()),
        StructField('payment_timestamp', TimestampType()),
        StructField('payment_status', IntegerType()),
        StructField('payment_method', StringType())
    ])
)

In [0]:
df_payment = (
    spark.read.format('csv')
    .option('sep', ',')
    .option('header', False)
    .schema(df_payment_schema)
    .load('/Volumes/gizmobox/landing/external_data/payments/*.csv')
)
display(df_payment)

In [0]:
df_payment.writeTo('gizmobox.bronze.py_payments').createOrReplace()

In [0]:
%sql
CREATE OR REPLACE TABLE gizmobox.bronze.py_customers 
AS
SELECT 
  input_file_name() as file_path
  ,current_timestamp() as date_run_created
  ,*
FROM json.`/Volumes/gizmobox/landing/operational_data/customers/*`


In [0]:
df_customers = spark.table('gizmobox.bronze.py_customers')
df_customers_trated = (
    df_customers
    .withColumn("date_run_created", F.from_utc_timestamp(F.col("date_run_created"), "America/Sao_Paulo"))
    .withColumn("date", F.date_format(F.col("date_run_created"), "yyyy-MM-dd"))
    .withColumn("hours", F.date_format(F.col("date_run_created"), "HH:mm:ss"))
    .drop("date_run_created")
)

display(df_customers_trated)

In [0]:
df_customers_trated.writeTo('gizmobox.silver.customers').createOrReplace()